In [3]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import squarify
sns.set(style="whitegrid")
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.metrics import log_loss 
from sklearn.model_selection import train_test_split
#pd.set_option('display.float_format', '{:.2f}'.format)

In [95]:
#df = pd.read_csv('dataTP1.csv')
test = pd.read_csv('Test_TP2_Datos_2020-2C.csv')
train = pd.read_csv('Train_TP2_Datos_2020-2C.csv')

In [96]:
train.columns

Index(['ID', 'Region', 'Territory', 'Pricing, Delivery_Terms_Quote_Appr',
       'Pricing, Delivery_Terms_Approved', 'Bureaucratic_Code_0_Approval',
       'Bureaucratic_Code_0_Approved', 'Submitted_for_Approval',
       'Bureaucratic_Code', 'Account_Created_Date', 'Source ',
       'Billing_Country', 'Account_Name', 'Opportunity_Name', 'Opportunity_ID',
       'Sales_Contract_No', 'Account_Owner', 'Opportunity_Owner',
       'Account_Type', 'Opportunity_Type', 'Quote_Type', 'Delivery_Terms',
       'Opportunity_Created_Date', 'Brand', 'Product_Type', 'Size',
       'Product_Category_B', 'Price', 'Currency', 'Last_Activity',
       'Quote_Expiry_Date', 'Last_Modified_Date', 'Last_Modified_By',
       'Product_Family', 'Product_Name', 'ASP_Currency', 'ASP',
       'ASP_(converted)_Currency', 'ASP_(converted)',
       'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date', 'Month',
       'Delivery_Quarter', 'Delivery_Year', 'Actual_Delivery_Date', 'TRF',
       'Total_Amount_Currenc

In [97]:
test.columns

Index(['ID', 'Region', 'Territory', 'Pricing, Delivery_Terms_Quote_Appr',
       'Pricing, Delivery_Terms_Approved', 'Bureaucratic_Code_0_Approval',
       'Bureaucratic_Code_0_Approved', 'Submitted_for_Approval',
       'Bureaucratic_Code', 'Account_Created_Date', 'Source ',
       'Billing_Country', 'Account_Name', 'Opportunity_Name', 'Opportunity_ID',
       'Sales_Contract_No', 'Account_Owner', 'Opportunity_Owner',
       'Account_Type', 'Opportunity_Type', 'Quote_Type', 'Delivery_Terms',
       'Opportunity_Created_Date', 'Brand', 'Product_Type', 'Size',
       'Product_Category_B', 'Price', 'Currency', 'Last_Activity',
       'Quote_Expiry_Date', 'Last_Modified_Date', 'Last_Modified_By',
       'Product_Family', 'Product_Name', 'ASP_Currency', 'ASP',
       'ASP_(converted)_Currency', 'ASP_(converted)',
       'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date', 'Month',
       'Delivery_Quarter', 'Delivery_Year', 'Actual_Delivery_Date', 'TRF',
       'Total_Amount_Currenc

In [98]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16947 entries, 0 to 16946
Data columns (total 52 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID                                  16947 non-null  int64  
 1   Region                              16947 non-null  object 
 2   Territory                           16947 non-null  object 
 3   Pricing, Delivery_Terms_Quote_Appr  16947 non-null  int64  
 4   Pricing, Delivery_Terms_Approved    16947 non-null  int64  
 5   Bureaucratic_Code_0_Approval        16947 non-null  int64  
 6   Bureaucratic_Code_0_Approved        16947 non-null  int64  
 7   Submitted_for_Approval              16947 non-null  int64  
 8   Bureaucratic_Code                   16947 non-null  object 
 9   Account_Created_Date                16947 non-null  object 
 10  Source                              16947 non-null  object 
 11  Billing_Country                     16947

In [99]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2551 entries, 0 to 2550
Data columns (total 51 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID                                  2551 non-null   int64  
 1   Region                              2551 non-null   object 
 2   Territory                           2551 non-null   object 
 3   Pricing, Delivery_Terms_Quote_Appr  2551 non-null   int64  
 4   Pricing, Delivery_Terms_Approved    2551 non-null   int64  
 5   Bureaucratic_Code_0_Approval        2551 non-null   int64  
 6   Bureaucratic_Code_0_Approved        2551 non-null   int64  
 7   Submitted_for_Approval              2551 non-null   int64  
 8   Bureaucratic_Code                   2551 non-null   object 
 9   Account_Created_Date                2551 non-null   object 
 10  Source                              2551 non-null   object 
 11  Billing_Country                     2551 no

* Contamos con un total de:
> * 16947 Registros en nuestro set de datos de *train*. 
> * 2551 Registros en nuestro set de datos de *test*.
> * Los 2551 registros a predecir en test, representan un 15% en cantidad de registros que tenemos en train (16947). En cantidad de oportunidades como son los porcentajes?

* Cuántas Oportunidades distintas tengo en *train* y *test*?

In [100]:
len(set(train.Opportunity_Name))

9841

In [101]:
len(set(test.Opportunity_Name))

1567

> * Se observa que las oportunidades a predecir en el set de *test* representan un 15,9% en cantidad de oportunidades que tenemos en *train*, manteniendo casi la misma proporción que teníamos con la cantidad de registros.

 * Veamos si existen registros duplicadas y para eso comparamos todos los valores existentes cada una de las filas sin tener en cuenta la columna *ID* que es el id único del registro o fila.

In [102]:
train[train.duplicated(subset=list(train.columns)[1:],keep=False)].shape[0]

619

In [103]:
test[test.duplicated(subset=list(test.columns)[1:],keep=False)].shape[0]

187

* Tenemos 619 Registros de *train* y 187 Registros de *test* que se repiten en nuestro set de datos.

* Respecto a esto: en nuestro set de *test* no podemos eliminar duplicados ya que son a todos esos registros a los que debemos predecir el *Stage*. En cambio en *train* podría pasar que filtrando estos casos repetidos podamos llegar a tener una mejor performance en nuestras predicciones ya que estaré entrenando con registros distintos y saco duplicados que no aportarían a priori información al entrenamiento sino mas bien algo de ruido. (Probar y en el Informe comentar si hay alguna mejora filtrando duplicados)

* Si nos quedamos en *train* con solo con la primera aparición de esos Registros duplicados, cuantos Registros en total debería filtrar?

In [104]:
train[train.duplicated(subset=list(train.columns)[1:],keep='first')].shape[0]

448

* A cuántas oportunidades distintas corresponden esos registros duplicados?

In [105]:
len(set(train[train.duplicated(subset=list(train.columns)[1:],keep=False)].Opportunity_Name))

131

In [106]:
# Vamos a eliminar duplicados excepto los Registros que correspondad a la primera aparición.
#train = train.drop_duplicates(subset=list(train.columns)[1:],keep='first') 
#len(set(train.ID))

* Si realizamos el filtrado de Registros repetidos en *train*, conservamos las mismas cantidades de Oportunidades (9841) pero pasamos de 16947 a 16499 Registros, osea que eliminamos 448 Registros duplicados que corresponden a un 2,64% de los registros iniciales. (Probar performance con este filtro)

### Limpieza de Regiones y Territorios 
##### (*la limpieza aplicada en train debe ser la misma que en test*)

In [107]:
train.Region.value_counts()

Japan          4892
EMEA           4664
Americas       3945
APAC           3262
Middle East     184
Name: Region, dtype: int64

In [108]:
test.Region.value_counts()

Americas       1016
EMEA            838
APAC            482
Japan           201
Middle East      14
Name: Region, dtype: int64

In [109]:
# Para los casos en que tenga a Japon como Region y a None como Territory le asigno Japón a Territory, sinó lo dejo como está
train['Territory'] = train.apply(lambda row: row['Region'] if ((row['Territory']=='None') & (row['Region']=='Japan')) else row['Territory'],axis=1)
test['Territory'] = test.apply(lambda row: row['Region'] if ((row['Territory']=='None') & (row['Region']=='Japan')) else row['Territory'],axis=1)

In [110]:
# Dejo Japón solo como territorio y como Región le asigno APAC (Asia-Pacífico)
train.loc[train['Region']=='Japan','Region']='Asia-Pacífico'
test.loc[test['Region']=='Japan','Region']='Asia-Pacífico'

In [111]:
# Cambio el nombre de la región Americas por América
train.loc[train['Region']=='Americas','Region']='América'
# Cambio el nombre de las demás Regiones
train.loc[train['Region']=='EMEA','Region']='Europa'
train.loc[train['Region']=='Middle East','Region']='Oriente Medio'
train.loc[train['Region']=='APAC','Region']='Asia-Pacífico'

# Lo mismo con test
test.loc[test['Region']=='Americas','Region']='América'
# Cambio el nombre de las demás Regiones
test.loc[test['Region']=='EMEA','Region']='Europa'
test.loc[test['Region']=='Middle East','Region']='Oriente Medio'
test.loc[test['Region']=='APAC','Region']='Asia-Pacífico'

In [112]:
train.Region.value_counts()

Asia-Pacífico    8154
Europa           4664
América          3945
Oriente Medio     184
Name: Region, dtype: int64

In [113]:
test.Region.value_counts()

América          1016
Europa            838
Asia-Pacífico     683
Oriente Medio      14
Name: Region, dtype: int64

* Un dato interesante a destacar es que en train la región predominante es Asia-Pacífico y en test en América siendo Asia-Pacífico la que ocupa la tercera posición. Este dato no es menor ya que puede que según cada región puedo sectorizar los entrenemientos y predicciones. Vamos a probar esta segmentación de predicciones para ver si obtenemos mejor performance que entrenar todo junto.

In [114]:
## Arreglamos la asignación de los territorios a sus correspondientes Regiones
train.loc[train['Territory']=='Saudi Arabia','Territory']= 'Arabia Saudita'
train.loc[train['Territory']=='KSA','Territory']= 'Arabia Saudita'
train.loc[(train['Territory']=='Arabia Saudita')|(train['Territory']=='Jordan')|(train['Territory']=='UAE (Dubai)')\
       |(train['Territory']=='Egypt')|(train['Territory']=='Kuwait'),'Region']= 'Oriente Medio'
train.loc[(train['Territory']=='South America')|(train['Territory']=='SE America')|(train['Territory']=='SW America')\
       |(train['Territory']=='Central America'),'Region']= 'América'
train.loc[train['Territory']=='Kazakhstan','Region']= 'Asia-Pacífico'

## Lo mismo para test
test.loc[test['Territory']=='Saudi Arabia','Territory']= 'Arabia Saudita'
test.loc[test['Territory']=='KSA','Territory']= 'Arabia Saudita'
test.loc[(test['Territory']=='Arabia Saudita')|(test['Territory']=='Jordan')|(test['Territory']=='UAE (Dubai)')\
       |(test['Territory']=='Egypt')|(test['Territory']=='Kuwait'),'Region']= 'Oriente Medio'
test.loc[(test['Territory']=='South America')|(test['Territory']=='SE America')|(test['Territory']=='SW America')\
       |(test['Territory']=='Central America'),'Region']= 'América'
test.loc[test['Territory']=='Kazakhstan','Region']= 'Asia-Pacífico'

### Agregamos el Hemisferio según corresponda a cada Territorio.

In [115]:
# Voy a Clasificar los Territorios según los Hemisferios que pertenecen
mis_kw_train = train['Territory']
mis_kw_test = test['Territory']

from flashtext import KeywordProcessor
keyword_processor = KeywordProcessor()
keyword_dict = {"Hemisferio Norte": ["NW America","Japan", "India","Italy","Germany","United Kingdom","UAE (Dubai)","Greece"\
                                     ,"Switzerland","Austria","NE America","Belgium","Tunisia",'SE America',"France","Spain","Singapore"\
                                     ,"Philippines","SW America",'Denmark', 'Central America', 'Poland', 'Norway', 'Thailand'\
                                     ,'Portugal', 'South Korea','Netherlands','Kuwait','Arabia Saudita','Cambodia', 'China (PRC)', 'KSA', 'Ireland'\
                                     , 'Jordan','Cyprus', 'Morocco', 'Turkey', 'Sweden', 'Monaco', 'Kazakhstan','Romania', 'Hungary'\
                                     , 'Taiwan','Macedonia', 'Ukraine', 'Czech Republic', 'Belarus','Vietnam', 'Saudi Arabia'\
                                     ,'Bosnia & Herzegovina', 'Kosovo', 'Luxembourg', 'Bulgaria','UAE (Others)', 'Armenia', 'Croatia'\
                                     , 'Malaysia', 'Georgia','Estonia', 'Pakistan', 'Canada', 'Central US', 'Carribean','Malta'\
                                     , 'Albania', 'Oman', 'Burma', 'Slovenia','Egypt']\
               ,"Hemisferio Sur": ["Australia",'South America','Indonesia','South East Asia','Solomon Islands','New Zealand']\
             }
key_depuradas_train=[]
key_depuradas_test=[]
keyword_processor.add_keywords_from_dict(keyword_dict)

for w in mis_kw_train:
       if len(keyword_processor.extract_keywords(w))==1:
            key_depuradas_train.append(keyword_processor.extract_keywords(w)[0])
       
       else: 
            key_depuradas_train.append(w)
            
for w in mis_kw_test:
       if len(keyword_processor.extract_keywords(w))==1:
            key_depuradas_test.append(keyword_processor.extract_keywords(w)[0])
       
       else: 
            key_depuradas_test.append(w)     

In [116]:
len(key_depuradas_train)

16947

In [117]:
len(key_depuradas_test)

2551

In [118]:
len(train['Territory'])

16947

In [119]:
len(test['Territory'])

2551

In [120]:
train['Hemisferio'] = key_depuradas_train
test['Hemisferio'] = key_depuradas_test

In [121]:
### Analizar hemisferios
#¿Filtro los hemisferios para los cuales no tenía un territorio definido?
#df = df.loc[(df['Hemisferio']!='None'),:]

In [122]:
train[['ID','Region','Territory','Hemisferio']].tail()

,ID,Region,Territory,Hemisferio
16942,8781,Europa,Austria,Hemisferio Norte
16943,8786,Europa,Austria,Hemisferio Norte
16944,8792,Europa,Austria,Hemisferio Norte
16945,28561,América,NE America,Hemisferio Norte
16946,28318,América,NE America,Hemisferio Norte


In [123]:
test[['ID','Region','Territory','Hemisferio']].tail()

,ID,Region,Territory,Hemisferio
2546,16345,Oriente Medio,Arabia Saudita,Hemisferio Norte
2547,15218,América,SE America,Hemisferio Norte
2548,15224,América,SE America,Hemisferio Norte
2549,7286,América,NE America,Hemisferio Norte
2550,11263,Europa,Belgium,Hemisferio Norte


### Agregamos La estación del año en que se crean las oportunidades en funcion del Hemisferio del Terrirorio al que pertenecen.

> * (podría agregar estaciones de entrega o de ultima modificación para tener más features)

In [124]:
train['Opportunity_Created_Date']=pd.to_datetime(train['Opportunity_Created_Date'])
test['Opportunity_Created_Date']=pd.to_datetime(test['Opportunity_Created_Date'])

In [125]:
def encuentra_Estacion_N(date): 
    year = str(date.year)
    seasons = {'Primavera': pd.date_range(start='21/03/'+year, end='20/06/'+year),
               'Verano': pd.date_range(start='21/06/'+year, end='20/09/'+year),
               'Otoño': pd.date_range(start='21/09/'+year, end='20/12/'+year)}
    if date in seasons['Primavera']:
        return 'Primavera'
    if date in seasons['Verano']:
        return 'Verano'
    if date in seasons['Otoño']:
        return 'Otoño'
    else:
        return 'Invierno'   
    
def encuentra_Estacion_S(date): 
    year = str(date.year)
    seasons = {'Otoño': pd.date_range(start='21/03/'+year, end='20/06/'+year),
               'Invierno': pd.date_range(start='21/06/'+year, end='20/09/'+year),
               'Primavera': pd.date_range(start='21/09/'+year, end='20/12/'+year)}
    if date in seasons['Invierno']:
        return 'Invierno'
    if date in seasons['Primavera']:
        return 'Primavera'
    if date in seasons['Otoño']:
        return 'Otoño'
    else:
        return 'Verano'     

In [126]:
train.loc[train['Hemisferio']=='Hemisferio Norte','Estación'] = train.loc[train['Hemisferio']=='Hemisferio Norte','Opportunity_Created_Date']\
.map(encuentra_Estacion_N)
train.loc[train['Hemisferio']=='Hemisferio Sur','Estación'] = train.loc[train['Hemisferio']=='Hemisferio Sur','Opportunity_Created_Date']\
.map(encuentra_Estacion_S)

test.loc[test['Hemisferio']=='Hemisferio Norte','Estación'] = test.loc[test['Hemisferio']=='Hemisferio Norte','Opportunity_Created_Date']\
.map(encuentra_Estacion_N)
test.loc[test['Hemisferio']=='Hemisferio Sur','Estación'] = test.loc[test['Hemisferio']=='Hemisferio Sur','Opportunity_Created_Date']\
.map(encuentra_Estacion_S)

In [127]:
train.Estación.value_counts(normalize=True)

Primavera    0.269047
Otoño        0.266696
Verano       0.234264
Invierno     0.229993
Name: Estación, dtype: float64

In [128]:
test.Estación.value_counts(normalize=True)

Invierno     0.676210
Primavera    0.233065
Otoño        0.052823
Verano       0.037903
Name: Estación, dtype: float64

In [129]:
train.shape

(16947, 54)

### Precio de las oportunidades

* Vamos a crear un feature con el valor total de ventas de las oportunidades (**Total_Taxable_Amount**) en una misma moneda (**dólares estadounidenses**) para que la comparación entre valores sean coherentes. También cabe destacar que en *test* tenemos casos de **Total_Taxable_Amount = 0** pero no tenemos casos de **Total_Amount = 0**, con lo cual calcularemos el precio final de cada **oportunidad** sumando dichos valores de Total_Amount para cada una de ellas. Esto lo vamos a realizar también en *train* pero tener en cuenta que train además cuenta con registros que contienen simultaneamente **Total_Taxable_Amount y Total_Taxable_Amount igual a 0**, (específicamente 39 casos que no se dan en test) con lo cual debo tomar la mejor decición, ya sea filtrando esos casos de train o imputando algún valor a Total_Taxable_Amount.    

In [130]:
train.loc[(train['Total_Amount']==0) & ((train['Total_Taxable_Amount']==0)),:]

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Hemisferio,Estación
464,28260,América,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/30/2015,...,NaT,0,USD,0.0,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno
579,25679,Asia-Pacífico,Japan,1,0,0,0,0,Bureaucratic_Code_5,4/20/2015,...,NaT,0,USD,0.0,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otoño
580,25718,Asia-Pacífico,Japan,1,0,0,0,0,Bureaucratic_Code_5,4/20/2015,...,NaT,0,USD,0.0,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otoño
586,15710,Asia-Pacífico,Japan,1,0,0,0,0,Bureaucratic_Code_5,12/9/2016,...,NaT,0,USD,0.0,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otoño
1130,17845,Europa,Germany,1,0,1,0,0,Bureaucratic_Code_5,10/29/2013,...,NaT,0,EUR,0.0,EUR,0.0,Closed Won,Prod_Category_A_None,Hemisferio Norte,Invierno
1437,22806,América,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,NaT,0,USD,0.0,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno
1705,14772,Oriente Medio,None,1,0,1,0,0,Bureaucratic_Code_5,11/15/2016,...,NaT,0,USD,0.0,USD,0.0,Closed Lost,Prod_Category_A_None,None,NaN
1904,8897,Europa,United Kingdom,1,0,1,0,0,Bureaucratic_Code_5,1/20/2016,...,NaT,5,EUR,0.0,EUR,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno
2083,10670,América,SE America,1,1,1,1,0,Bureaucratic_Code_4,2/13/2016,...,NaT,0,USD,0.0,USD,0.0,Closed Won,Prod_Category_A_None,Hemisferio Norte,Invierno
2823,14860,Europa,None,1,0,1,0,0,Bureaucratic_Code_5,5/9/2016,...,NaT,5,USD,0.0,USD,0.0,Closed Lost,Prod_Category_A_None,None,NaN


In [131]:
train = train.loc[(train['Total_Amount']!=0) | ((train['Total_Taxable_Amount']!=0)),:]

In [132]:
train.shape

(16908, 54)

In [133]:
test.loc[test['Total_Amount']==0,:]

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Prod_Category_A,Hemisferio,Estación


In [134]:
test.loc[(test['Total_Amount']==0) & ((test['Total_Taxable_Amount']==0)),:]

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Prod_Category_A,Hemisferio,Estación


In [135]:
train.iloc[186:188]

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Hemisferio,Estación
186,8142,América,NE America,1,1,1,1,0,Bureaucratic_Code_4,12/24/2015,...,NaT,1,USD,659835.0,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno
187,8148,América,NE America,1,1,1,1,0,Bureaucratic_Code_4,12/24/2015,...,NaT,1,USD,672512.5,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno


In [136]:
train_monto_total=train.groupby('Opportunity_ID').agg({'Total_Amount':'sum'}).reset_index()
train_monto_total.rename(columns={'Total_Amount':'Total_moneda_local'},inplace=True)
train_monto_total.head()

,Opportunity_ID,Total_moneda_local
0,0,5272800.0
1,1,48230.0
2,2,83865.6
3,3,7421881.5
4,4,13357192.5


In [137]:
test_monto_total=test.groupby('Opportunity_ID').agg({'Total_Amount':'sum'}).reset_index()
test_monto_total.rename(columns={'Total_Amount':'Total_moneda_local'},inplace=True)
test_monto_total.head()

,Opportunity_ID,Total_moneda_local
0,10689,367419.0
1,10690,757783.5
2,10691,21037.5
3,10692,2169106.5
4,10693,5752.5


In [138]:
train=pd.merge(train,train_monto_total,how='inner',on='Opportunity_ID')

In [139]:
test=pd.merge(test,test_monto_total,how='inner',on='Opportunity_ID')

In [140]:
train.iloc[186:188]

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Hemisferio,Estación,Total_moneda_local
186,8142,América,NE America,1,1,1,1,0,Bureaucratic_Code_4,12/24/2015,...,1,USD,659835.0,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno,1332347.5
187,8148,América,NE America,1,1,1,1,0,Bureaucratic_Code_4,12/24/2015,...,1,USD,672512.5,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno,1332347.5


In [141]:
train.ASP_Currency.value_counts()

USD    7338
JPY    4520
EUR    4489
AUD     556
GBP       5
Name: ASP_Currency, dtype: int64

In [142]:
train.loc[train['ASP_Currency']=='GBP',:]

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Hemisferio,Estación,Total_moneda_local
2290,20861,Europa,United Kingdom,1,1,1,1,0,Bureaucratic_Code_4,6/16/2014,...,5,GBP,1837080.00,GBP,7.274250e+06,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno,7274250.00
2291,20878,Europa,United Kingdom,1,1,1,1,0,Bureaucratic_Code_4,6/16/2014,...,5,GBP,1801800.00,GBP,7.274250e+06,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno,7274250.00
2292,20881,Europa,United Kingdom,1,1,1,1,0,Bureaucratic_Code_4,6/16/2014,...,5,GBP,1798290.00,GBP,7.274250e+06,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno,7274250.00
2293,20884,Europa,United Kingdom,1,1,1,1,0,Bureaucratic_Code_4,6/16/2014,...,5,GBP,1837080.00,GBP,7.274250e+06,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno,7274250.00
3565,17651,Europa,United Kingdom,1,1,1,1,0,Bureaucratic_Code_4,10/15/2013,...,2,GBP,776081.25,EUR,9.043541e+05,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Verano,776081.25


In [143]:
train.head()

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Hemisferio,Estación,Total_moneda_local
0,27761,Europa,None,1,1,1,1,0,Bureaucratic_Code_4,6/16/2015,...,10,EUR,5272800.0,EUR,5272800.0,Closed Lost,Prod_Category_A_None,None,NaN,5272800.0
1,27760,Europa,None,0,0,0,0,0,Bureaucratic_Code_4,6/16/2015,...,0,EUR,48230.0,EUR,48230.0,Closed Won,Prod_Category_A_None,None,NaN,48230.0
2,27446,América,NW America,0,0,0,0,0,Bureaucratic_Code_4,4/21/2015,...,0,USD,83865.6,USD,83865.6,Closed Won,Prod_Category_A_None,Hemisferio Norte,Otoño,83865.6
3,16808,América,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,14,USD,7421881.5,USD,7421881.5,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otoño,7421881.5
4,16805,América,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,25,USD,13357192.5,USD,13357192.5,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otoño,13357192.5


In [144]:
def conver_EUR_dolares(monto): 
        return monto*1.131   
    
def conver_JPY_dolares(monto): 
        return monto*0.00899 

def conver_AUD_dolares(monto): 
        return monto*0.7076 
    
def conver_GBP_dolares(monto): 
        return monto*1.3181 

def conver_USD_dolares(monto): 
        return monto*1

In [145]:
train.loc[train['Total_Taxable_Amount_Currency']=='EUR','total_en_dolares'] = train.loc[train['Total_Taxable_Amount_Currency']=='EUR','Total_moneda_local']\
.map(conver_EUR_dolares)
train.loc[train['Total_Taxable_Amount_Currency']=='JPY','total_en_dolares'] = train.loc[train['Total_Taxable_Amount_Currency']=='JPY','Total_moneda_local']\
.map(conver_JPY_dolares)
train.loc[train['Total_Taxable_Amount_Currency']=='AUD','total_en_dolares'] = train.loc[train['Total_Taxable_Amount_Currency']=='AUD','Total_moneda_local']\
.map(conver_AUD_dolares)
train.loc[train['Total_Taxable_Amount_Currency']=='GBP','total_en_dolares'] = train.loc[train['Total_Taxable_Amount_Currency']=='GBP','Total_moneda_local']\
.map(conver_GBP_dolares)
train.loc[train['Total_Taxable_Amount_Currency']=='USD','total_en_dolares'] = train.loc[train['Total_Taxable_Amount_Currency']=='USD','Total_moneda_local']\
.map(conver_USD_dolares)

In [146]:
test.loc[test['Total_Taxable_Amount_Currency']=='EUR','total_en_dolares'] = test.loc[test['Total_Taxable_Amount_Currency']=='EUR','Total_moneda_local']\
.map(conver_EUR_dolares)
test.loc[test['Total_Taxable_Amount_Currency']=='JPY','total_en_dolares'] = test.loc[test['Total_Taxable_Amount_Currency']=='JPY','Total_moneda_local']\
.map(conver_JPY_dolares)
test.loc[test['Total_Taxable_Amount_Currency']=='AUD','total_en_dolares'] = test.loc[test['Total_Taxable_Amount_Currency']=='AUD','Total_moneda_local']\
.map(conver_AUD_dolares)
test.loc[test['Total_Taxable_Amount_Currency']=='GBP','total_en_dolares'] = test.loc[test['Total_Taxable_Amount_Currency']=='GBP','Total_moneda_local']\
.map(conver_GBP_dolares)
test.loc[test['Total_Taxable_Amount_Currency']=='USD','total_en_dolares'] = test.loc[test['Total_Taxable_Amount_Currency']=='USD','Total_moneda_local']\
.map(conver_USD_dolares)

### Total Amount en dólares

In [147]:
train.loc[train['Total_Amount_Currency']=='EUR','monto_en_dolares'] = train.loc[train['Total_Amount_Currency']=='EUR','Total_Amount']\
.map(conver_EUR_dolares)
train.loc[train['Total_Amount_Currency']=='JPY','monto_en_dolares'] = train.loc[train['Total_Amount_Currency']=='JPY','Total_Amount']\
.map(conver_JPY_dolares)
train.loc[train['Total_Amount_Currency']=='AUD','monto_en_dolares'] = train.loc[train['Total_Amount_Currency']=='AUD','Total_Amount']\
.map(conver_AUD_dolares)
train.loc[train['Total_Amount_Currency']=='GBP','monto_en_dolares'] = train.loc[train['Total_Amount_Currency']=='GBP','Total_Amount']\
.map(conver_GBP_dolares)
train.loc[train['Total_Amount_Currency']=='USD','monto_en_dolares'] = train.loc[train['Total_Amount_Currency']=='USD','Total_Amount']\
.map(conver_USD_dolares)

test.loc[test['Total_Amount_Currency']=='EUR','monto_en_dolares'] = test.loc[test['Total_Amount_Currency']=='EUR','Total_Amount']\
.map(conver_EUR_dolares)
test.loc[test['Total_Amount_Currency']=='JPY','monto_en_dolares'] = test.loc[test['Total_Amount_Currency']=='JPY','Total_Amount']\
.map(conver_JPY_dolares)
test.loc[test['Total_Amount_Currency']=='AUD','monto_en_dolares'] = test.loc[test['Total_Amount_Currency']=='AUD','Total_Amount']\
.map(conver_AUD_dolares)
test.loc[test['Total_Amount_Currency']=='GBP','monto_en_dolares'] = test.loc[test['Total_Amount_Currency']=='GBP','Total_Amount']\
.map(conver_GBP_dolares)
test.loc[test['Total_Amount_Currency']=='USD','monto_en_dolares'] = test.loc[test['Total_Amount_Currency']=='USD','Total_Amount']\
.map(conver_USD_dolares)

In [148]:
train.head(20)

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Hemisferio,Estación,Total_moneda_local,total_en_dolares,monto_en_dolares
0,27761,Europa,None,1,1,1,1,0,Bureaucratic_Code_4,6/16/2015,...,5272800.0,EUR,5272800.0,Closed Lost,Prod_Category_A_None,None,NaN,5272800.0,5.963537e+06,5.963537e+06
1,27760,Europa,None,0,0,0,0,0,Bureaucratic_Code_4,6/16/2015,...,48230.0,EUR,48230.0,Closed Won,Prod_Category_A_None,None,NaN,48230.0,5.454813e+04,5.454813e+04
2,27446,América,NW America,0,0,0,0,0,Bureaucratic_Code_4,4/21/2015,...,83865.6,USD,83865.6,Closed Won,Prod_Category_A_None,Hemisferio Norte,Otoño,83865.6,8.386560e+04,8.386560e+04
3,16808,América,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,7421881.5,USD,7421881.5,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otoño,7421881.5,7.421882e+06,7.421882e+06
4,16805,América,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,13357192.5,USD,13357192.5,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otoño,13357192.5,1.335719e+07,1.335719e+07
5,16802,América,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,14838277.5,USD,14838277.5,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otoño,14838277.5,1.483828e+07,1.483828e+07
6,16799,América,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,2659494.6,USD,2659494.6,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otoño,2659494.6,2.659495e+06,2.659495e+06
7,27455,América,NW America,1,1,1,1,0,Bureaucratic_Code_4,4/21/2015,...,50688.0,USD,50688.0,Closed Won,Prod_Category_A_None,Hemisferio Norte,Otoño,50688.0,5.068800e+04,5.068800e+04
8,24353,Asia-Pacífico,Japan,1,0,0,0,0,Bureaucratic_Code_5,4/20/2015,...,15600.0,JPY,470400.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otoño,470400.0,4.228896e+03,1.402440e+02
9,24355,Asia-Pacífico,Japan,1,0,0,0,0,Bureaucratic_Code_5,4/20/2015,...,4400.0,JPY,470400.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otoño,470400.0,4.228896e+03,3.955600e+01


In [149]:
test.head(10)

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Prod_Category_A,Hemisferio,Estación,Total_moneda_local,total_en_dolares,monto_en_dolares
0,6140,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,EUR,162240.0,EUR,367419.0,Prod_Category_A_None,Hemisferio Norte,Primavera,367419.0,415550.889,183493.44
1,6146,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,EUR,78624.0,EUR,367419.0,Prod_Category_A_None,Hemisferio Norte,Primavera,367419.0,415550.889,88923.744
2,6151,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,EUR,126555.0,EUR,367419.0,Prod_Category_A_None,Hemisferio Norte,Primavera,367419.0,415550.889,143133.705
3,6118,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,EUR,243360.0,EUR,757783.5,Prod_Category_A_None,Hemisferio Norte,Primavera,757783.5,857053.1385,275240.16
4,6124,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,EUR,157248.0,EUR,757783.5,Prod_Category_A_None,Hemisferio Norte,Primavera,757783.5,857053.1385,177847.488
5,6129,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,EUR,126555.0,EUR,757783.5,Prod_Category_A_None,Hemisferio Norte,Primavera,757783.5,857053.1385,143133.705
6,6135,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,EUR,118156.5,EUR,757783.5,Prod_Category_A_None,Hemisferio Norte,Primavera,757783.5,857053.1385,133635.0015
7,6218,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,EUR,112464.0,EUR,757783.5,Prod_Category_A_None,Hemisferio Norte,Primavera,757783.5,857053.1385,127196.784
8,11530,América,NE America,1,1,0,0,0,Bureaucratic_Code_4,2/16/2019,...,USD,21037.5,USD,21037.5,Prod_Category_A_None,Hemisferio Norte,Primavera,21037.5,21037.5,21037.5
9,20314,América,NW America,1,1,1,0,0,Bureaucratic_Code_2,7/27/2013,...,USD,342490.5,USD,2169106.5,Prod_Category_A_None,Hemisferio Norte,Primavera,2169106.5,2169106.5,342490.5


In [150]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16908 entries, 0 to 16907
Data columns (total 57 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   ID                                  16908 non-null  int64         
 1   Region                              16908 non-null  object        
 2   Territory                           16908 non-null  object        
 3   Pricing, Delivery_Terms_Quote_Appr  16908 non-null  int64         
 4   Pricing, Delivery_Terms_Approved    16908 non-null  int64         
 5   Bureaucratic_Code_0_Approval        16908 non-null  int64         
 6   Bureaucratic_Code_0_Approved        16908 non-null  int64         
 7   Submitted_for_Approval              16908 non-null  int64         
 8   Bureaucratic_Code                   16908 non-null  object        
 9   Account_Created_Date                16908 non-null  object        
 10  Source                

In [151]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2551 entries, 0 to 2550
Data columns (total 56 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   ID                                  2551 non-null   int64         
 1   Region                              2551 non-null   object        
 2   Territory                           2551 non-null   object        
 3   Pricing, Delivery_Terms_Quote_Appr  2551 non-null   int64         
 4   Pricing, Delivery_Terms_Approved    2551 non-null   int64         
 5   Bureaucratic_Code_0_Approval        2551 non-null   int64         
 6   Bureaucratic_Code_0_Approved        2551 non-null   int64         
 7   Submitted_for_Approval              2551 non-null   int64         
 8   Bureaucratic_Code                   2551 non-null   object        
 9   Account_Created_Date                2551 non-null   object        
 10  Source                  

In [152]:
train.groupby('Opportunity_ID').agg({'monto_en_dolares':['max','min','mean','std']}).reset_index()

Opportunity_ID monto_en_dolares                              \
                                 max           min          mean   
0                 0       5963536.80  5.963537e+06  5.963537e+06   
1                 1         54548.13  5.454813e+04  5.454813e+04   
2                 2         83865.60  8.386560e+04  8.386560e+04   
3                 3       7421881.50  7.421882e+06  7.421882e+06   
4                 4      13357192.50  1.335719e+07  1.335719e+07   
...             ...              ...           ...           ...   
9800          12799        454322.70  4.543227e+05  4.543227e+05   
9801          12800      10751580.00  1.058092e+07  1.066625e+07   
9802          12801        116888.85  1.052000e+05  1.129926e+05   
9803          12802       2346796.88  2.346797e+06  2.346797e+06   
9804          12803      25603200.00  2.560320e+07  2.560320e+07   

                     
                std  
0               NaN  
1               NaN  
2               NaN  
3               NaN  
4               NaN  
...             ...  
9800            NaN  
9801  120674.843277  
9802    6748.580901  
9803            NaN  
9804            NaN  

[9805 rows x 5 columns]

In [153]:
test.total_en_dolares=test.total_en_dolares.astype('float')
test.monto_en_dolares=test.monto_en_dolares.astype('float')

In [154]:
train_monto=train.groupby('Opportunity_ID').agg({'monto_en_dolares':['max','min','mean','std']}).reset_index()
test_monto=test.groupby('Opportunity_ID').agg({'monto_en_dolares':['max','min','mean','std']}).reset_index()

In [155]:
train_monto

Opportunity_ID monto_en_dolares                              \
                                 max           min          mean   
0                 0       5963536.80  5.963537e+06  5.963537e+06   
1                 1         54548.13  5.454813e+04  5.454813e+04   
2                 2         83865.60  8.386560e+04  8.386560e+04   
3                 3       7421881.50  7.421882e+06  7.421882e+06   
4                 4      13357192.50  1.335719e+07  1.335719e+07   
...             ...              ...           ...           ...   
9800          12799        454322.70  4.543227e+05  4.543227e+05   
9801          12800      10751580.00  1.058092e+07  1.066625e+07   
9802          12801        116888.85  1.052000e+05  1.129926e+05   
9803          12802       2346796.88  2.346797e+06  2.346797e+06   
9804          12803      25603200.00  2.560320e+07  2.560320e+07   

                     
                std  
0               NaN  
1               NaN  
2               NaN  
3               NaN  
4               NaN  
...             ...  
9800            NaN  
9801  120674.843277  
9802    6748.580901  
9803            NaN  
9804            NaN  

[9805 rows x 5 columns]

In [156]:
test_monto

Opportunity_ID monto_en_dolares                                        
                                 max          min         mean           std
0             10689      183493.4400   88923.7440  138516.9630  47453.583431
1             10690      275240.1600  127196.7840  171410.6277  61244.287928
2             10691       21037.5000   21037.5000   21037.5000           NaN
3             10692      456654.0000  228327.0000  361517.7500  85939.160738
4             10693        5752.5000    5752.5000    5752.5000           NaN
...             ...              ...          ...          ...           ...
1562          12364      124740.0000   23010.0000   73875.0000  71933.972850
1563          12365       45054.9000   45054.9000   45054.9000           NaN
1564          12366      100122.0000  100122.0000  100122.0000           NaN
1565          12367      143220.0000  143220.0000  143220.0000           NaN
1566          12368      985124.5248  985124.5248  985124.5248           NaN

[1567 rows x 5 columns]

In [157]:
train_monto.columns

MultiIndex([(  'Opportunity_ID',     ''),
            ('monto_en_dolares',  'max'),
            ('monto_en_dolares',  'min'),
            ('monto_en_dolares', 'mean'),
            ('monto_en_dolares',  'std')],
           )

In [158]:
train_monto.columns = ['Opportunity_ID','monto_en_dolares_max','monto_en_dolares_min','monto_en_dolares_mean','monto_en_dolares_std']
test_monto.columns = ['Opportunity_ID','monto_en_dolares_max','monto_en_dolares_min','monto_en_dolares_mean','monto_en_dolares_std']

In [159]:
train=pd.merge(train,train_monto,how='inner',on='Opportunity_ID')
test=pd.merge(test,test_monto,how='inner',on='Opportunity_ID')

In [160]:
train.head()

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Prod_Category_A,Hemisferio,Estación,Total_moneda_local,total_en_dolares,monto_en_dolares,monto_en_dolares_max,monto_en_dolares_min,monto_en_dolares_mean,monto_en_dolares_std
0,27761,Europa,None,1,1,1,1,0,Bureaucratic_Code_4,6/16/2015,...,Prod_Category_A_None,None,NaN,5272800.0,5963536.80,5963536.80,5963536.80,5963536.80,5963536.80,NaN
1,27760,Europa,None,0,0,0,0,0,Bureaucratic_Code_4,6/16/2015,...,Prod_Category_A_None,None,NaN,48230.0,54548.13,54548.13,54548.13,54548.13,54548.13,NaN
2,27446,América,NW America,0,0,0,0,0,Bureaucratic_Code_4,4/21/2015,...,Prod_Category_A_None,Hemisferio Norte,Otoño,83865.6,83865.60,83865.60,83865.60,83865.60,83865.60,NaN
3,16808,América,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,Prod_Category_A_None,Hemisferio Norte,Otoño,7421881.5,7421881.50,7421881.50,7421881.50,7421881.50,7421881.50,NaN
4,16805,América,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,Prod_Category_A_None,Hemisferio Norte,Otoño,13357192.5,13357192.50,13357192.50,13357192.50,13357192.50,13357192.50,NaN


In [161]:
test.head()

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Prod_Category_A,Hemisferio,Estación,Total_moneda_local,total_en_dolares,monto_en_dolares,monto_en_dolares_max,monto_en_dolares_min,monto_en_dolares_mean,monto_en_dolares_std
0,6140,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,Prod_Category_A_None,Hemisferio Norte,Primavera,367419.0,415550.8890,183493.440,183493.44,88923.744,138516.9630,47453.583431
1,6146,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,Prod_Category_A_None,Hemisferio Norte,Primavera,367419.0,415550.8890,88923.744,183493.44,88923.744,138516.9630,47453.583431
2,6151,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,Prod_Category_A_None,Hemisferio Norte,Primavera,367419.0,415550.8890,143133.705,183493.44,88923.744,138516.9630,47453.583431
3,6118,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,Prod_Category_A_None,Hemisferio Norte,Primavera,757783.5,857053.1385,275240.160,275240.16,127196.784,171410.6277,61244.287928
4,6124,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,Prod_Category_A_None,Hemisferio Norte,Primavera,757783.5,857053.1385,177847.488,275240.16,127196.784,171410.6277,61244.287928


### Sales_Contract_No es equivalente a Stage y no se permite utilizar este feature!!!! (dejo comentado la demostración gráfica de que son casi lo mismo)

In [69]:
train.Sales_Contract_No.value_counts().head()

None    6938
108       36
479       36
2575      18
3123      16
Name: Sales_Contract_No, dtype: int64

In [70]:
train['contrato']='Sin Contrato'
test['contrato']='Sin Contrato'

In [71]:
train.loc[train['Sales_Contract_No'].isnull(),:]

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Hemisferio,Estación,Total_moneda_local,total_en_dolares,monto_en_dolares,monto_en_dolares_max,monto_en_dolares_min,monto_en_dolares_mean,monto_en_dolares_std,contrato


In [72]:
train.loc[train['Sales_Contract_No']!='None','contrato']='Con Contrato'
test.loc[test['Sales_Contract_No']!='None','contrato']='Con Contrato'

In [73]:
train.contrato.value_counts()

Con Contrato    9970
Sin Contrato    6938
Name: contrato, dtype: int64

In [74]:
#train=train.loc[(train['Stage']!='Proposal') & (train['Stage']!='Negotiation') & (train['Stage']!='Qualification'),:].copy()

In [75]:
 '''train['veces'] = 1
mi_map=pd.crosstab(train['contrato'], train['Stage'],values=train.veces,aggfunc=np.sum)

fig, ax = plt.subplots(figsize=(10,10))
g = sns.heatmap(mi_map, cmap="viridis", ax=ax, linewidths=0.005, linecolor='black',annot=True,fmt='g')
g.set_title("Cantidad de Oportunidades Exitosas y No Exitosas según si hubo o no Contrato", fontsize=16)
g.set_xlabel("Tipo de Stage", fontsize=16, labelpad=10)
g.set_ylabel("Región", fontsize=16, labelpad=16, rotation=90)
g.set_xticklabels(g.get_xticklabels(), fontsize=13)
g.set_yticklabels(g.get_yticklabels(), fontsize=13, rotation=0)
g.title.set_position([0.5, 1.05]);
#plt.savefig('graficos/7.png');
plt.show();'''

'train[\'veces\'] = 1\nmi_map=pd.crosstab(train[\'contrato\'], train[\'Stage\'],values=train.veces,aggfunc=np.sum)\n\nfig, ax = plt.subplots(figsize=(10,10))\ng = sns.heatmap(mi_map, cmap="viridis", ax=ax, linewidths=0.005, linecolor=\'black\',annot=True,fmt=\'g\')\ng.set_title("Cantidad de Oportunidades Exitosas y No Exitosas según si hubo o no Contrato", fontsize=16)\ng.set_xlabel("Tipo de Stage", fontsize=16, labelpad=10)\ng.set_ylabel("Región", fontsize=16, labelpad=16, rotation=90)\ng.set_xticklabels(g.get_xticklabels(), fontsize=13)\ng.set_yticklabels(g.get_yticklabels(), fontsize=13, rotation=0)\ng.title.set_position([0.5, 1.05]);\n#plt.savefig(\'graficos/7.png\');\nplt.show();'

### Stage

In [162]:
train.Stage.value_counts()

Closed Won       9529
Closed Lost      7315
Proposal           35
Negotiation        18
Qualification      11
Name: Stage, dtype: int64

* En el set de datos de *train* se observa que en la variable Stage, además de los estados que indican si una oportunidad resultó exitosa o no (**estados finales**), tenemos otros 3 estados (Proposal, Negotiation y Qualification) que corresponden a **estados intermedios** que tienen las oportunidades dentro del pipeline que desarrolla la empresa.

* Como con esta variable se estará luego tratando de predecir la “Probabilidad de éxito” para cada oportunidad en el set de *test*, o sea ¿Cuál es la probabilidad de que la oportunidad se convierta en un caso Closed Won? . Que decisión debemos tomar con dichos estados? Dichos estados al no tener una resolución, debemos asignarle arbitrariamente algún estado final? Debemos filtrar estos casos?

> * Finalmente luego de analizar esos registros con estos estados intermedios y verificar que corresponen a oportunidades que no tuvieron aún un desenlase (exitoso o no exitoso) y tenindo en cuenta que corresponden a unos pocos casos, tomamos la decisión de filtrar estos registros y quedarnos solamente con los que tienen un estado final definido.

In [163]:
train=train.loc[(train['Stage']!='Proposal') & (train['Stage']!='Negotiation') & (train['Stage']!='Qualification'),:]

In [164]:
train.Stage.value_counts()

Closed Won     9529
Closed Lost    7315
Name: Stage, dtype: int64

### Cantidad de Items de cada Oportunidad

In [165]:
Op_cant_train = train.groupby(['Opportunity_ID'], as_index=False)['ID'].count()
Op_cant_test = test.groupby(['Opportunity_ID'], as_index=False)['ID'].count()
Op_cant_train.rename(columns = {'ID': 'Cantidad de Items Asociados'}, inplace = True)
Op_cant_test.rename(columns = {'ID': 'Cantidad de Items Asociados'}, inplace = True)

train=pd.merge(Op_cant_train,train,on='Opportunity_ID')
test=pd.merge(Op_cant_test,test,on='Opportunity_ID')

In [166]:
train[['ID','Region','Opportunity_ID','Cantidad de Items Asociados']].sample(5)

,ID,Region,Opportunity_ID,Cantidad de Items Asociados
4798,10919,Europa,2856,3
8195,9816,Asia-Pacífico,5147,2
1933,20074,Europa,1155,1
12387,18471,Europa,7560,2
12020,10210,Europa,7350,1


In [167]:
test[['ID','Region','Opportunity_ID','Cantidad de Items Asociados']].sample(5)

,ID,Region,Opportunity_ID,Cantidad de Items Asociados
364,9314,Europa,10903,1
943,20405,Europa,11368,2
836,10352,Asia-Pacífico,11289,1
2095,20755,Europa,12098,1
2479,17187,Europa,12311,3


In [168]:
train.head(20)

,Opportunity_ID,Cantidad de Items Asociados,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,...,Prod_Category_A,Hemisferio,Estación,Total_moneda_local,total_en_dolares,monto_en_dolares,monto_en_dolares_max,monto_en_dolares_min,monto_en_dolares_mean,monto_en_dolares_std
0,0,1,27761,Europa,None,1,1,1,1,0,...,Prod_Category_A_None,None,NaN,5272800.0,5.963537e+06,5.963537e+06,5.963537e+06,5.963537e+06,5.963537e+06,NaN
1,1,1,27760,Europa,None,0,0,0,0,0,...,Prod_Category_A_None,None,NaN,48230.0,5.454813e+04,5.454813e+04,5.454813e+04,5.454813e+04,5.454813e+04,NaN
2,2,1,27446,América,NW America,0,0,0,0,0,...,Prod_Category_A_None,Hemisferio Norte,Otoño,83865.6,8.386560e+04,8.386560e+04,8.386560e+04,8.386560e+04,8.386560e+04,NaN
3,3,1,16808,América,NW America,1,0,1,0,0,...,Prod_Category_A_None,Hemisferio Norte,Otoño,7421881.5,7.421882e+06,7.421882e+06,7.421882e+06,7.421882e+06,7.421882e+06,NaN
4,4,1,16805,América,NW America,1,0,1,0,0,...,Prod_Category_A_None,Hemisferio Norte,Otoño,13357192.5,1.335719e+07,1.335719e+07,1.335719e+07,1.335719e+07,1.335719e+07,NaN
5,5,1,16802,América,NW America,1,0,1,0,0,...,Prod_Category_A_None,Hemisferio Norte,Otoño,14838277.5,1.483828e+07,1.483828e+07,1.483828e+07,1.483828e+07,1.483828e+07,NaN
6,6,1,16799,América,NW America,1,0,1,0,0,...,Prod_Category_A_None,Hemisferio Norte,Otoño,2659494.6,2.659495e+06,2.659495e+06,2.659495e+06,2.659495e+06,2.659495e+06,NaN
7,7,1,27455,América,NW America,1,1,1,1,0,...,Prod_Category_A_None,Hemisferio Norte,Otoño,50688.0,5.068800e+04,5.068800e+04,5.068800e+04,5.068800e+04,5.068800e+04,NaN
8,8,5,24353,Asia-Pacífico,Japan,1,0,0,0,0,...,Prod_Category_A_None,Hemisferio Norte,Otoño,470400.0,4.228896e+03,1.402440e+02,2.734758e+03,3.955600e+01,8.457792e+02,1173.733656
9,8,5,24355,Asia-Pacífico,Japan,1,0,0,0,0,...,Prod_Category_A_None,Hemisferio Norte,Otoño,470400.0,4.228896e+03,3.955600e+01,2.734758e+03,3.955600e+01,8.457792e+02,1173.733656


### Precio total en dolares por Items de la Oportunidad

In [169]:
train['precio_por_items'] = train['total_en_dolares']/train['Cantidad de Items Asociados']

In [170]:
test['precio_por_items'] = test['total_en_dolares']/test['Cantidad de Items Asociados']

In [171]:
test.head()

,Opportunity_ID,Cantidad de Items Asociados,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,...,Hemisferio,Estación,Total_moneda_local,total_en_dolares,monto_en_dolares,monto_en_dolares_max,monto_en_dolares_min,monto_en_dolares_mean,monto_en_dolares_std,precio_por_items
0,10689,3,6140,Europa,Germany,1,1,1,1,0,...,Hemisferio Norte,Primavera,367419.0,415550.8890,183493.440,183493.44,88923.744,138516.9630,47453.583431,138516.9630
1,10689,3,6146,Europa,Germany,1,1,1,1,0,...,Hemisferio Norte,Primavera,367419.0,415550.8890,88923.744,183493.44,88923.744,138516.9630,47453.583431,138516.9630
2,10689,3,6151,Europa,Germany,1,1,1,1,0,...,Hemisferio Norte,Primavera,367419.0,415550.8890,143133.705,183493.44,88923.744,138516.9630,47453.583431,138516.9630
3,10690,5,6118,Europa,Germany,1,1,1,1,0,...,Hemisferio Norte,Primavera,757783.5,857053.1385,275240.160,275240.16,127196.784,171410.6277,61244.287928,171410.6277
4,10690,5,6124,Europa,Germany,1,1,1,1,0,...,Hemisferio Norte,Primavera,757783.5,857053.1385,177847.488,275240.16,127196.784,171410.6277,61244.287928,171410.6277


### Media del Total Amount en dolares por items de Oportunidad

In [172]:
train['media_por_items'] = train['monto_en_dolares_mean']/train['Cantidad de Items Asociados']
test['media_por_items'] = test['monto_en_dolares_mean']/test['Cantidad de Items Asociados']

### Fechas (Faltaría calcular, tiempos medios, máximos y minimos por oportunidad a las diferencias temporales obtenidas)

In [85]:
train['Opportunity_Created_Date']=pd.to_datetime(train['Opportunity_Created_Date'])
test['Opportunity_Created_Date']=pd.to_datetime(test['Opportunity_Created_Date'])

train['Last_Activity']=pd.to_datetime(train['Last_Activity'])
test['Last_Activity']=pd.to_datetime(test['Last_Activity'])

train['Quote_Expiry_Date']=pd.to_datetime(train['Quote_Expiry_Date'])
test['Quote_Expiry_Date']=pd.to_datetime(test['Quote_Expiry_Date'])

train['Last_Modified_Date']=pd.to_datetime(train['Last_Modified_Date'])
test['Last_Modified_Date']=pd.to_datetime(test['Last_Modified_Date'])

train['Planned_Delivery_Start_Date']=pd.to_datetime(train['Planned_Delivery_Start_Date'])
test['Planned_Delivery_Start_Date']=pd.to_datetime(test['Planned_Delivery_Start_Date'])

train['Planned_Delivery_End_Date']=pd.to_datetime(train['Planned_Delivery_End_Date'])
test['Planned_Delivery_End_Date']=pd.to_datetime(test['Planned_Delivery_End_Date'])

train['Opportunity_Created_Date']=pd.to_datetime(train['Opportunity_Created_Date'])
test['Opportunity_Created_Date']=pd.to_datetime(test['Opportunity_Created_Date'])

In [86]:
train['year_entrega']=train['Planned_Delivery_End_Date'].dt.year
train['mes_entrega']=train['Planned_Delivery_End_Date'].dt.month
test['year_entrega']=test['Planned_Delivery_End_Date'].dt.year
test['mes_entrega']=test['Planned_Delivery_End_Date'].dt.month

In [87]:
train.head(15)

,Opportunity_ID,Cantidad de Items Asociados,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,...,total_en_dolares,monto_en_dolares,monto_en_dolares_max,monto_en_dolares_min,monto_en_dolares_mean,monto_en_dolares_std,precio_por_items,media_por_items,year_entrega,mes_entrega
0,0,1,27761,Europa,None,1,1,1,1,0,...,5.963537e+06,5.963537e+06,5.963537e+06,5.963537e+06,5.963537e+06,NaN,5.963537e+06,5.963537e+06,2016.0,6.0
1,1,1,27760,Europa,None,0,0,0,0,0,...,5.454813e+04,5.454813e+04,5.454813e+04,5.454813e+04,5.454813e+04,NaN,5.454813e+04,5.454813e+04,2016.0,1.0
2,2,1,27446,América,NW America,0,0,0,0,0,...,8.386560e+04,8.386560e+04,8.386560e+04,8.386560e+04,8.386560e+04,NaN,8.386560e+04,8.386560e+04,2016.0,1.0
3,3,1,16808,América,NW America,1,0,1,0,0,...,7.421882e+06,7.421882e+06,7.421882e+06,7.421882e+06,7.421882e+06,NaN,7.421882e+06,7.421882e+06,2018.0,3.0
4,4,1,16805,América,NW America,1,0,1,0,0,...,1.335719e+07,1.335719e+07,1.335719e+07,1.335719e+07,1.335719e+07,NaN,1.335719e+07,1.335719e+07,2018.0,2.0
5,5,1,16802,América,NW America,1,0,1,0,0,...,1.483828e+07,1.483828e+07,1.483828e+07,1.483828e+07,1.483828e+07,NaN,1.483828e+07,1.483828e+07,2018.0,3.0
6,6,1,16799,América,NW America,1,0,1,0,0,...,2.659495e+06,2.659495e+06,2.659495e+06,2.659495e+06,2.659495e+06,NaN,2.659495e+06,2.659495e+06,2017.0,5.0
7,7,1,27455,América,NW America,1,1,1,1,0,...,5.068800e+04,5.068800e+04,5.068800e+04,5.068800e+04,5.068800e+04,NaN,5.068800e+04,5.068800e+04,2016.0,1.0
8,8,5,24353,Asia-Pacífico,Japan,1,0,0,0,0,...,4.228896e+03,1.402440e+02,2.734758e+03,3.955600e+01,8.457792e+02,1173.733656,8.457792e+02,1.691558e+02,2016.0,2.0
9,8,5,24355,Asia-Pacífico,Japan,1,0,0,0,0,...,4.228896e+03,3.955600e+01,2.734758e+03,3.955600e+01,8.457792e+02,1173.733656,8.457792e+02,1.691558e+02,2016.0,2.0


In [88]:
train['tiempo_creacion_ulti_mod']=train['Last_Modified_Date']-train['Opportunity_Created_Date']
train['tiempo_expiracion_ulti_mod']=train['Last_Modified_Date']-train['Quote_Expiry_Date']
train['tiempo_plan_end_star']=train['Planned_Delivery_End_Date']-train['Planned_Delivery_Start_Date']

In [89]:
test['tiempo_creacion_ulti_mod']=test['Last_Modified_Date']-test['Opportunity_Created_Date']
test['tiempo_expiracion_ulti_mod']=test['Last_Modified_Date']-test['Quote_Expiry_Date']
test['tiempo_plan_end_star']=test['Planned_Delivery_End_Date']-test['Planned_Delivery_Start_Date']

In [90]:
train.head()

,Opportunity_ID,Cantidad de Items Asociados,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,...,monto_en_dolares_min,monto_en_dolares_mean,monto_en_dolares_std,precio_por_items,media_por_items,year_entrega,mes_entrega,tiempo_creacion_ulti_mod,tiempo_expiracion_ulti_mod,tiempo_plan_end_star
0,0,1,27761,Europa,None,1,1,1,1,0,...,5963536.80,5963536.80,NaN,5963536.80,5963536.80,2016.0,6.0,189 days,135 days,60 days
1,1,1,27760,Europa,None,0,0,0,0,0,...,54548.13,54548.13,NaN,54548.13,54548.13,2016.0,1.0,39 days,-5 days,2 days
2,2,1,27446,América,NW America,0,0,0,0,0,...,83865.60,83865.60,NaN,83865.60,83865.60,2016.0,1.0,296 days,-1 days,0 days
3,3,1,16808,América,NW America,1,0,1,0,0,...,7421881.50,7421881.50,NaN,7421881.50,7421881.50,2018.0,3.0,840 days,NaT,58 days
4,4,1,16805,América,NW America,1,0,1,0,0,...,13357192.50,13357192.50,NaN,13357192.50,13357192.50,2018.0,2.0,840 days,298 days,27 days


In [91]:
train['tiempo_creacion_ulti_mod']=train['tiempo_creacion_ulti_mod'].dt.total_seconds()/36000

In [92]:
train['tiempo_expiracion_ulti_mod'] = train['tiempo_expiracion_ulti_mod'].dt.total_seconds()/36000
train['tiempo_plan_end_star'] = train['tiempo_plan_end_star'].dt.total_seconds()/36000

In [93]:
train.head()

,Opportunity_ID,Cantidad de Items Asociados,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,...,monto_en_dolares_min,monto_en_dolares_mean,monto_en_dolares_std,precio_por_items,media_por_items,year_entrega,mes_entrega,tiempo_creacion_ulti_mod,tiempo_expiracion_ulti_mod,tiempo_plan_end_star
0,0,1,27761,Europa,None,1,1,1,1,0,...,5963536.80,5963536.80,NaN,5963536.80,5963536.80,2016.0,6.0,453.6,324.0,144.0
1,1,1,27760,Europa,None,0,0,0,0,0,...,54548.13,54548.13,NaN,54548.13,54548.13,2016.0,1.0,93.6,-12.0,4.8
2,2,1,27446,América,NW America,0,0,0,0,0,...,83865.60,83865.60,NaN,83865.60,83865.60,2016.0,1.0,710.4,-2.4,0.0
3,3,1,16808,América,NW America,1,0,1,0,0,...,7421881.50,7421881.50,NaN,7421881.50,7421881.50,2018.0,3.0,2016.0,NaN,139.2
4,4,1,16805,América,NW America,1,0,1,0,0,...,13357192.50,13357192.50,NaN,13357192.50,13357192.50,2018.0,2.0,2016.0,715.2,64.8


In [94]:
test['tiempo_creacion_ulti_mod']=test['tiempo_creacion_ulti_mod'].dt.total_seconds()/36000
test['tiempo_expiracion_ulti_mod'] = test['tiempo_expiracion_ulti_mod'].dt.total_seconds()/36000
test['tiempo_plan_end_star'] = test['tiempo_plan_end_star'].dt.total_seconds()/36000

In [97]:
test.head(20)

,Opportunity_ID,Cantidad de Items Asociados,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,...,monto_en_dolares_mean,monto_en_dolares_std,contrato,precio_por_items,media_por_items,year_entrega,mes_entrega,tiempo_creacion_ulti_mod,tiempo_expiracion_ulti_mod,tiempo_plan_end_star
0,10689,3,6140,Europa,Germany,1,1,1,1,0,...,138516.9630,47453.583431,Con Contrato,138516.9630,46172.321000,2019.0,5.0,0.0,-43.2,72.0
1,10689,3,6146,Europa,Germany,1,1,1,1,0,...,138516.9630,47453.583431,Con Contrato,138516.9630,46172.321000,2019.0,5.0,0.0,-43.2,72.0
2,10689,3,6151,Europa,Germany,1,1,1,1,0,...,138516.9630,47453.583431,Con Contrato,138516.9630,46172.321000,2019.0,5.0,0.0,-43.2,72.0
3,10690,5,6118,Europa,Germany,1,1,1,1,0,...,171410.6277,61244.287928,Con Contrato,171410.6277,34282.125540,2019.0,6.0,31.2,-12.0,64.8
4,10690,5,6124,Europa,Germany,1,1,1,1,0,...,171410.6277,61244.287928,Con Contrato,171410.6277,34282.125540,2019.0,6.0,31.2,-12.0,67.2
5,10690,5,6129,Europa,Germany,1,1,1,1,0,...,171410.6277,61244.287928,Con Contrato,171410.6277,34282.125540,2019.0,6.0,31.2,-12.0,67.2
6,10690,5,6135,Europa,Germany,1,1,1,1,0,...,171410.6277,61244.287928,Con Contrato,171410.6277,34282.125540,2019.0,6.0,31.2,-12.0,64.8
7,10690,5,6218,Europa,Germany,1,1,1,1,0,...,171410.6277,61244.287928,Con Contrato,171410.6277,34282.125540,2019.0,6.0,31.2,-12.0,64.8
8,10691,1,11530,América,NE America,1,1,0,0,0,...,21037.5000,NaN,Con Contrato,21037.5000,21037.500000,2019.0,10.0,9.6,NaN,0.0
9,10692,6,20314,América,NW America,1,1,1,0,0,...,361517.7500,85939.160738,Con Contrato,361517.7500,60252.958333,2019.0,12.0,9.6,NaN,0.0


In [175]:
len(set(train.Opportunity_Name))

9755

In [174]:
len(set(test.Opportunity_Name))

1567

In [173]:
train.to_csv("train_listo.csv", index=False)
test.to_csv("test_listo.csv", index=False)

In [69]:
train['pred']=0

In [70]:
train.pred.value_counts()

0    16883
Name: pred, dtype: int64

In [75]:
train.loc[train['Stage']=='Closed Won','pred']=1
train.loc[train['Stage']!='Closed Won','pred']=0

In [76]:
train.pred.value_counts()

1    9533
0    7350
Name: pred, dtype: int64

In [79]:
train['target']=1

In [80]:
log_loss(train.pred, train.target)

15.036775664497156

In [81]:
train = train.drop_duplicates(subset='Opportunity_ID',keep='first') 

In [83]:
log_loss(train.pred, train.target)

16.64715137451298